In [1]:
import pkg.dbs.postgres
from pkg import config
from pkg import claude
import json
import os

In [2]:
api_key = config.get_API_keys()

In [3]:
# Get DB Schema
conf = config.getConfig("./config/config.ini")
pconf = conf['postgres']
psql = pkg.dbs.postgres.PSQL(pconf['host'], int(pconf['port']),pconf['db'],pconf['user'],pconf['pw'])

# Tables
tables = psql.get_db_schema()

# Foreign Keys
foreign_keys = psql.get_foregin_keys()

# Types
types = psql.get_db_types()

# Initialize with your API key
generator = claude.SQLQueryGenerator(api_key)


# Define example Q&A pairs
examples = [
    {
        "question": "How many people whose name is Hong Gil-dong are there?",
        "query": "SELECT count(*) FROM 인적정보 WHERE Name = '홍길동';"
    },
    {
        "question": "How many certificates have the name 통신자격증 are there?",
        "query": "SELECT count(*) FROM 자격증 WHERE Name = '통신자격증';"
    },
    {
        "question": "How many ReqGrade 2 are there?",
        "query": "SELECT count(*) FROM 병과 WHERE ReqGrade = 2;"
    }
]


# Generate query for a sample question
question = '"서울특별시"에 거주하는 사람의 정보를 모두 보고 싶어'
result = generator.generate_query(question, tables, examples, foreign_keys, types)

# Print the result
print("Generated SQL Query:")
print(result.query if result.query else result.explanation)


Generated SQL Query:
SELECT * FROM personalinformation WHERE address1 LIKE '%서울특별시%';


In [4]:
print(result)

query="SELECT * FROM personalinformation WHERE address1 LIKE '%서울특별시%';" explanation=None


In [5]:
query_res = psql.query(result.query)

In [6]:
data = json.dumps(query_res, ensure_ascii=False)

In [7]:
report_generator = claude.ReportGenerator(api_key)
report = report_generator.generate_report(data=data,
                                 question=question,
                                 explanation=result.explanation
                                 )

In [11]:
report_res = json.loads(report.content)

In [14]:
print(report_res['Answer'])

요청하신 '서울특별시'에 거주하는 사람들의 정보를 제공해 드리겠습니다. 제공된 데이터는 주민등록번호, 이름, 주소, 종교, 학력, 직업, 자녀수, 건강보험종류 등의 개인 정보를 포함하고 있습니다. 총 506명의 정보가 포함되어 있으며, 이들은 모두 서울특별시에 거주하고 있습니다. 일부 데이터에는 누락된 부분도 있습니다. 이 데이터를 바탕으로 서울특별시 거주민들의 다양한 인구통계학적 특성을 분석할 수 있을 것 같습니다.
